# <center>Ch4 企业级微调_模型量化</center>

# 背景

### 1. 大模型规模爆炸性增长
自从Transformer架构出现后，语言模型的规模呈指数级增长。从BERT的3.4亿参数，到GPT-3的1750亿参数，再到GPT-4和Claude等超大规模模型，参数量可能已达数万亿级别。这种增长带来性能提升的同时，也造成了巨大的资源消耗。    
大模型量化和蒸馏就是把一个"大而强但很慢很贵"的AI模型变成"小而快但依然够用"的技术。   

### 2. 计算资源与成本压力
训练和部署大模型需要昂贵的硬件设施：
- GPU/TPU集群成本高昂
- 云计算资源费用持续累积
- 电力消耗巨大
- 维护大型计算集群的人力成本

### 3. 终端设备的局限性
消费级设备（手机、笔记本、IoT设备等）无法直接运行大型AI模型：
- 存储空间有限
- 计算能力不足
- 电池续航问题
- 发热问题

### 4. 实时性需求
许多应用场景对响应速度有严格要求：
- 在线翻译
- 语音助手
- 自动驾驶
- 实时内容生成与推荐

### 5. 隐私与安全考量
将数据发送到云端处理存在隐私风险：
- 个人敏感信息可能泄露
- 企业数据安全隐患
- 合规性要求（如GDPR、CCPA等）

 ## 大模型量化与蒸馏的必要性

基于上述背景，大模型量化与蒸馏成为解决这些问题的关键技术：

1. **弥合理论与实践的鸿沟**：大模型虽然强大，但实际应用受限，量化与蒸馏技术使其能力得以在现实环境中落地

2. **降低AI民主化门槛**：通过量化与蒸馏，使更多组织和个人能够负担得起AI技术应用，推动技术普及

3. **实现效能与性能的平衡**：在计算资源有限的情况下寻求最优的性能-资源平衡点

4. **满足特定场景需求**：针对性地为不同应用场景定制轻量级模型，如医疗、法律、金融等垂直领域

大模型量化与蒸馏技术的出现是AI领域从"追求极致性能"向"追求实际可用性"转变的重要标志，也是大模型时代技术成熟和商业化的必经之路。



## 什么是大模型量化技术？

大模型量化技术是将大模型（特别是大型语言模型、视觉模型等）中的高精度浮点数（通常为FP32或FP16）转换为低精度数值表示（如INT8、INT4等）的过程，旨在减少模型的计算和存储需求，同时尽可能保持模型性能。就像把高清照片转成压缩照片，虽然会损失一些细节，但可以节省很多空间。   

举例：原来模型中的权重值如0.7352可能被转换成整数7，配合一个缩放因子0.1，在使用时再还原为0.7。


## 为什么需要量化？
 
 想象一下，手机内存只有8GB，但一个原始模型需要12GB，这时你就无法使用。量化后，模型可能只需要3GB，就能在手机上运行了。     

量化的主要优势包括：

1. **模型体积减小**：INT8比FP32节省75%存储空间，INT4可节省87.5%
2. **推理速度提升**：低精度运算通常更快，许多硬件有专门的低精度加速
3. **内存占用降低**：减少运行时内存需求，使大模型能在受限设备上运行
4. **能耗降低**：低精度计算通常能耗更低，对移动设备和边缘设备尤为重要
5. **吞吐量提升**：可以在同样硬件上处理更多并发请求


## 适用场景

静态量化特别适用于以下场景：

1. **资源受限环境**：边缘设备、移动设备、消费级GPU，手机、平板等内存有限的设备等
2. **高吞吐量需求**：需要处理大量并发请求的服务
3. **延迟敏感应用**：需要快速响应的实时系统
4. **批量推理场景**：预测任务批量处理
5. **部署环境固定**：应用场景和数据分布相对稳定

## 与其他模型压缩技术的对比

| 技术 | 原理 | 优势 | 劣势 | 与量化的互补性 |
|------|------|------|------|--------------|
| **量化** | 降低数值精度 | 直接实现、性能提升明显、适用大多数模型 | 精度损失、可能需要微调 | - |
| **知识蒸馏** | 小模型学习大模型行为 | 模型体积和结构简化、适应性强 | 需要重新训练、蒸馏过程复杂 | 可先蒸馏后量化，双重压缩 |
| **剪枝** | 移除不重要连接/神经元 | 减少计算路径、保留关键结构 | 需专门硬件/软件支持才能发挥优势、训练复杂 | 常与量化结合使用 |
| **参数共享** | 多个权重使用同一参数 | 直接减少独立参数数量 | 可能显著降低模型表达能力 | 能与量化协同工作 |




## 量化技术的优缺点总结

### 优点
- 实现相对简单，大多数框架直接支持
- 不改变模型结构，易于集成到现有系统
- 显著减小模型体积（4-8倍）
- 硬件加速支持广泛（特别是INT8）
- 部署友好，无需专门训练

### 缺点
- 精度损失，尤其在高压缩比（如INT4）下
- 对某些任务（如长文本生成）影响较大
- 可能需要特定优化才能保持性能
- 不同于蒸馏，不减少模型结构复杂度


更多原理信息请查看 **《Ch8 大模型量化与蒸馏技术详解》**

# 1. 量化技术与框架

量化可以在模型生命周期的不同阶段进行，每个位置有其特点和适用场景。下面详细分析各种量化位置的区别及适用场景。

## 1.1 主要量化位置对比

### 1. 训练后量化(PTQ)
**位置**: 完成训练后，部署前

```json
预训练 → 微调 → [量化] → 部署
```

### 2. 量化感知训练(QAT)
**位置**: 训练/微调过程中

```json
预训练 → [量化感知微调] → 转换 → 部署
```

### 3. 量化后微调(QFT)
**位置**: 先量化，再微调

```json
预训练 → 微调 → 量化 → [微调] → 部署
```

### 4. 训练阶段量化(TTQ/QT)
**位置**: 完整训练过程中

```json
[低精度训练] → 导出量化模型 → 部署
```



## 2 详细比较与适用场景

| 方法                | 实施方式                                                                                     | 优势                                                                                     | 劣势                                                                                     | 适用场景                                                                                     | 实际应用例子                                                                                     |
|---------------------|----------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------|
| **1. 训练后量化(PTQ)** | - 在完全训练好的模型上直接应用量化<br>- 使用少量校准数据确定量化参数<br>- 不需要重新训练或微调               | - 实施简单快速<br>- 计算资源需求低<br>- 无需训练数据或仅需少量校准数据<br>- 可以快速尝试不同量化配置 | - 精度损失可能较大<br>- 对某些模型结构效果不佳<br>- 控制精度损失的能力有限                       | - 资源有限环境<br>- 时间紧迫场景<br>- 探索阶段<br>- 模型结构适合<br>- 通用模型部署                 | - 将通用大模型(如BERT/GPT)快速量化部署到线上服务<br>- 将视觉模型量化到移动设备<br>- 快速评估模型在低精度下的表现 |
| **2. 量化感知训练(QAT)** | - 在训练或微调过程中模拟量化操作<br>- 使用伪量化节点和直通估计器<br>- 模型学习补偿量化误差                   | - 精度损失最小<br>- 模型能适应量化带来的精度影响<br>- 可以追求极限压缩比(如INT4/INT2)             | - 需要完整训练/微调流程<br>- 计算资源需求高<br>- 训练时间长<br>- 实现复杂度高                    | - 性能极其敏感<br>- 极限压缩需求<br>- 资源充足<br>- 复杂模型<br>- 特定领域优化                     | - 将语音识别模型量化到4位精度而保持准确率<br>- 医疗诊断模型的高精度量化<br>- 自动驾驶感知模型的关键组件量化 |
| **3. 量化后微调(QFT)** | - 先对模型进行量化<br>- 再使用少量数据微调已量化的模型<br>- 针对量化后的精度损失进行恢复                   | - 比QAT资源需求低<br>- 比PTQ精度更高<br>- 可以针对特定任务优化<br>- 训练时间相对较短              | - 仍需要训练资源<br>- 需要适当的微调数据<br>- 实现稍复杂                                      | - 平衡场景<br>- 特定任务适应<br>- 有限数据场景<br>- 增量改善<br>- 部署后优化                       | - 将量化后的大语言模型针对客服场景微调<br>- 量化视觉模型后，针对新环境条件微调<br>- 解决量化后模型在特定输入上的性能下降 |
| **4. 训练阶段量化(TTQ/QT)** | - 从头开始就使用低精度训练<br>- 权重和激活值都使用低精度表示<br>- 设计特殊训练策略适应低精度                 | - 训练和推理一致<br>- 训练过程也能节省资源<br>- 理论上量化误差最小<br>- 适合从头训练的场景           | - 实现技术要求高<br>- 训练不稳定风险<br>- 需要专门优化的训练流程<br>- 框架支持可能不完善          | - 从头训练<br>- 极端资源限制<br>- 专用硬件<br>- 研究场景<br>- 一致性要求                         | - 在量化友好的专用硬件上训练模型<br>- 边缘AI系统的完整低精度训练流程<br>- 为资源受限环境设计的模型架构 |



## 3.量化相关名词概念

### **1. 量化算法 vs 量化精度**

**量化算法**（决定"怎么量化"）：
- **GPTQ**：基于二阶信息的量化算法
- **AWQ**：激活感知权重量化算法
- **SmoothQuant**：平滑量化算法
- **AQLM**：加法量化算法

**量化精度**（决定"量化到几位"）：
- **INT4**：4位整数精度
- **INT8**：8位整数精度  
- **FP16**：16位浮点精度

**组合关系**：
- GPTQ + INT4 = 用GPTQ算法量化到4位
- AWQ + INT4 = 用AWQ算法量化到4位
- 同一个算法可以量化到不同精度

### **2. 量化方法 vs 量化框架**

**量化方法**：具体的算法和技术
- **GPTQ**：一种量化算法
- **AWQ**：一种量化算法  
- **GGML/GGUF**：一种量化格式标准

**量化框架**：实现这些方法的工具库
- **GPTQModel**：实现GPTQ算法的框架
- **AutoGPTQ**：实现GPTQ算法的框架
- **llama.cpp**：实现GGML/GGUF格式的框架
- **bitsandbytes**：实现多种量化方法的框架

## 4.量化技术体系的完整架构

### **1. 框架与算法的关系**

| 框架 | 支持的算法 | 支持的精度 | 特点 |
|------|------------|------------|------|
| **bitsandbytes** | NF4、FP4、INT8 | 4位、8位 | 算法+精度打包，无需选择 |
| **GPTQModel** | GPTQ | INT4、INT8 | 单算法，支持多精度 (AutoGPTQ的官方继任者，解决了原版本的维护问题)|
| **AutoAWQ** | AWQ | INT4 | 单算法，专门优化 |
| **llama.cpp** | GGML/GGUF | 各种Q等级 | 专门的格式标准 |
| **Transformers** | 集成多种 | 各种组合 | 平台化，支持多种后端 |


### **2. HuggingFace上的量化格式解释**

| 文件格式 | 含义 | 精度 | 特点 |
|----------|------|------|------|
| `.gguf` | GGUF格式 | INT4/INT8 | CPU推理优化，llama.cpp使用 |
| `-int4` | INT4量化 | 4位整数 | 体积最小，精度损失较大 |
| `-int8` | INT8量化 | 8位整数 | 平衡体积和精度 |
| `-gptq` | GPTQ算法 | 主要INT4 | GPU推理优化 |
| `-awq` | AWQ算法 | 主要INT4 | GPU推理优化，精度更好 |
| `-fp16` | 半精度浮点 | 16位浮点 | 原始精度，体积较大 |

- **W4A16**：权重4位，激活16位
- **W8A8**：权重8位，激活8位  
- **W4A8**：权重4位，激活8位
- **W3A16**：权重3位，激活16位



### **3. 推理框架 vs 量化框架**

**推理框架**：
- **llama.cpp**：CPU推理框架
- **vLLM**：GPU推理框架
- **TensorRT-LLM**：NVIDIA GPU推理框架

**量化框架**：
- **GPTQModel**：实现GPTQ算法
- **AutoAWQ**：实现AWQ算法
- **bitsandbytes**：实现多种量化算法

**概念层次**：
1. **量化算法**：GPTQ、AWQ、SmoothQuant等（决定怎么量化）
2. **量化精度**：INT4、INT8、FP16等（决定量化到几位）
3. **量化框架**：GPTQModel、AutoAWQ等（实现算法的工具）
4. **推理框架**：vLLM、llama.cpp等（运行量化模型的平台）



### **4. 如何选择的系统性建议**

**选择决策树：**

```json
1. 部署环境？
   ├── CPU → GGUF (llama.cpp)
   ├── GPU → 继续下一步
   
2. 使用场景？
   ├── 生产环境 → GPTQ (GPTQModel)
   ├── 长上下文 → EXL2 (ExLlamaV2)  
   ├── 实验研究 → bitsandbytes
   
3. 精度要求？
   ├── 最高精度 → W8A8 或 W4A16
   ├── 平衡性能 → W4A16
   ├── 最小体积 → W4A8 或 W3A16
```

## 5.不同硬件设备的适配

**CPU优化方案：**
- **GGUF + llama.cpp**：专门为CPU推理优化
- **支持的精度**：Q2_K到Q8_0各种等级
- **优势**：CPU推理速度快，内存使用低

**GPU优化方案：**
- **GPTQ + GPTQModel**：GPU加速量化和推理
- **AWQ + AutoAWQ**：GPU推理优化
- **EXL2 + ExLlamaV2**：支持4位KV缓存，长上下文优化

**基于实际基准测试数据：**

| 设备类型 | 推荐方案 | 精度 | 理由 |
|----------|----------|------|------|
| **CPU (特别是Mac)** | GGUF + llama.cpp | Q4_K_M | CPU推理最优化 |
| **GPU (NVIDIA)** | GPTQ + GPTQModel | W4A16 | GPU加速，生产就绪 |
| **GPU (长上下文)** | EXL2 + ExLlamaV2 | W4A16 | 支持4位KV缓存 |
| **资源受限** | bitsandbytes | NF4 | 动态量化，内存友好 |

### **1.补充说明**

**llama.cpp/GGUF并不是只能用于CPU！**

**实际支持的设备：**
- **CPU**：主要优化目标，支持所有主流CPU架构
- **GPU**：支持NVIDIA GPU（CUDA）、AMD GPU（ROCm）、Intel GPU
- **Apple Silicon**：支持Mac M1/M2/M3的GPU加速（Metal）
- **其他加速器**：支持OpenCL等

**GPTQ和AWQ并不是只能用于GPU！**

**根据vLLM的官方硬件支持表：**
- **AWQ**：支持GPU（NVIDIA）、Intel GPU、**x86 CPU**
- **GPTQ**：支持GPU（NVIDIA）、Intel GPU、**x86 CPU**

### **2 各技术的实际设备支持**

| 技术 | CPU支持 | GPU支持 | 主要优化目标 | 性能特点 |
|------|---------|---------|-------------|----------|
| **GGUF + llama.cpp** | ✅ 主要优化 | ✅ 支持加速 | CPU推理 | CPU上最快，GPU可加速 |
| **GPTQ + GPTQModel** | ✅ 支持 | ✅ 主要优化 | GPU推理 | GPU上最快，CPU较慢 |
| **AWQ + AutoAWQ** | ✅ 支持 | ✅ 主要优化 | GPU推理 | GPU上最快，CPU较慢 |
| **bitsandbytes** | ❌ 不支持 | ✅ 仅GPU | GPU推理 | 仅GPU，动态量化 |


### **3 性能差异说明**

**GGUF在不同设备上的表现：**
- **CPU**：专门优化，性能最佳
- **GPU**：可以加速，但不如专门的GPU量化方法

**GPTQ/AWQ在不同设备上的表现：**
- **GPU**：专门优化，性能最佳
- **CPU**：可以运行，但性能较差



### **4 基于设备的推荐**

**如果主要使用CPU：**
- **首选**：GGUF + llama.cpp
- **备选**：GPTQ（性能较差）
- **不推荐**：AWQ、bitsandbytes

**如果主要使用GPU：**
- **首选**：GPTQ + GPTQModel 或 AWQ + AutoAWQ
- **备选**：GGUF + llama.cpp（可以GPU加速）
- **特殊用途**：bitsandbytes（动态量化）



### **4.边界和限制**

**技术边界：**
- **bitsandbytes**：真正只支持GPU
- **其他技术**：都支持CPU和GPU，只是优化程度不同

**性能边界：**
- **CPU优化**：GGUF > GPTQ > AWQ
- **GPU优化**：GPTQ/AWQ > GGUF > bitsandbytes


## 6.如何选择量化技术

### **1 主要推理框架**

| 推理框架 | 核心特点 | 主要优势 | 适用场景 |
|---------|----------|----------|----------|
| **vLLM** | 高吞吐量推理引擎 | PagedAttention、批处理优化 | 生产环境、高并发 |
| **SGLang** | 新一代推理引擎 | 性能优于vLLM，纯Python实现 | 高性能需求、易定制 |
| **llama.cpp** | 跨平台推理引擎 | CPU优化、跨平台支持 | 边缘设备、CPU推理 |
| **TensorRT-LLM** | NVIDIA官方引擎 | 极致性能优化 | NVIDIA GPU、高性能 |
| **Text-Generation-Inference (TGI)** | HuggingFace官方 | 易用性、生态完整 | 快速部署、原型开发 |



### **2 完整兼容性矩阵**

| 量化技术 | vLLM | SGLang | llama.cpp | TensorRT-LLM | TGI |
|----------|------|--------|-----------|--------------|-----|
| **GPTQ** | ✅ | ✅ | ❌ | ✅ | ✅ |
| **AWQ** | ✅ | ✅ | ❌ | ✅ | ❌ |
| **GGUF** | ❌ | ❌ | ✅ | ❌ | ❌ |
| **EXL2** | ❌ | ❌ | ❌ | ❌ | ❌ |
| **bitsandbytes** | ❌ | ❌ | ❌ | ❌ | ✅ |
| **FP8** | ✅ | ✅ | ❌ | ✅ | ❌ |
| **torchao** | ❌ | ✅ | ❌ | ❌ | ❌ |



### **3 各框架的量化支持详情**

**vLLM支持的量化：**
- GPTQ (4bit/8bit)
- AWQ (4bit)
- FP8 (通过CUTLASS)
- SqueezeLLM
- 不支持：GGUF、EXL2、bitsandbytes

**SGLang支持的量化：**
- GPTQ (4bit/8bit)
- AWQ (4bit)
- FP8 (通过CUTLASS)
- torchao (INT4/INT8/FP8) - 新增功能
- 不支持：GGUF、EXL2、bitsandbytes

**llama.cpp支持的量化：**
- GGUF格式的所有量化类型
- 不支持：GPTQ、AWQ、EXL2、bitsandbytes

**TensorRT-LLM支持的量化：**
- GPTQ (4bit/8bit)
- AWQ (4bit)
- FP8
- INT8/INT4权重量化
- 不支持：GGUF、EXL2、bitsandbytes

**TGI支持的量化：**
- GPTQ (4bit/8bit)
- bitsandbytes (4bit/8bit)
- 不支持：AWQ、GGUF、EXL2



### **5 推荐的技术栈组合**

**生产环境推荐：**
1. **GPTQ + vLLM** - 最成熟稳定
2. **AWQ + SGLang** - 最高性能
3. **GGUF + llama.cpp** - CPU场景

**开发测试推荐：**
1. **GPTQ + TGI** - 快速原型
2. **bitsandbytes + TGI** - 简单易用


# 2. 数据准备

GPTQ为什么需要激活数据

**你要"量化"一道菜的调料**

### **不使用激活数据的方式（错误方法）**
```json
厨师说："我要减少调料用量来节省成本"
看了看调料瓶：盐、糖、胡椒粉、味精都是白色粉末
决定：每种调料都减少一半用量
```

**结果**：菜变得很难吃，因为：
- 盐很重要，减少一半菜就没味道了
- 胡椒粉不太重要，减少一半影响不大
- 但厨师不知道哪个重要，所以全部减半



### **使用激活数据的方式（GPTQ方法）**
```json
厨师说："我要先试试每种调料对菜味道的影响"
实验过程：
1. 做一道标准菜（这就是"激活数据"）
2. 分别减少每种调料，尝尝味道变化
3. 发现：盐减少10%，菜就很淡；胡椒粉减少50%，几乎没影响
4. 决定：盐只减少5%，胡椒粉减少60%，总体还是节省了成本
```

**结果**：菜还是很好吃，因为重要的调料保持了足够的量



## 回到神经网络

### **权重就像调料**
- 神经网络有几十亿个权重参数
- 就像菜谱中有很多种调料
- 每个权重对最终结果的重要性不同

### **激活数据就像"试菜"**
- 把真实数据输入网络，看看每个权重的作用
- 就像厨师试菜来判断调料的重要性
- 通过这个过程知道哪些权重重要，哪些不重要

### **量化就像"减少调料"**
- 量化是把32位的权重变成4位（减少精度来节省内存）
- 就像减少调料用量来节省成本
- 如果随便减少，效果会很差



## 为什么必须要激活数据？

**因为你不知道哪些权重重要！**

### **详细解释**
```json
情况1：没有激活数据
- 看到权重A = 0.5，权重B = 0.5
- 不知道哪个重要，所以都量化成同样精度
- 结果：如果A很重要但被量化得很粗糙，网络性能就坏了

情况2：有激活数据
- 用真实数据测试发现：权重A影响很大，权重B影响很小
- 决定：权重A用高精度量化，权重B用低精度量化
- 结果：在相同的存储空间下，网络性能保持得很好
```



## 数字举例

假设你有两个权重：

**原始权重**：
- 权重A = 0.12345（很重要）
- 权重B = 0.67890（不重要）

**不用激活数据的量化**：
- 权重A量化为 0.12（损失了0.00345）
- 权重B量化为 0.68（损失了0.00890）
- 但是A的损失影响很大！

**用激活数据的量化**：
- 发现A重要，B不重要
- 权重A量化为 0.123（只损失0.00045）
- 权重B量化为 0.7（损失了0.01890，但影响小）
- 总体效果更好！


## 为什么是"必须"？

**因为神经网络太复杂了！**

- 人类无法直接看出哪个权重重要
- 有几十亿个权重，不可能一个个分析
- 只能通过运行真实数据来"测试"每个权重的作用
- 这就像厨师必须尝菜才知道调料的重要性一样

## **总结**

**GPTQ需要激活数据的原因**：
1. **识别重要性**：知道哪些权重重要，哪些不重要
2. **差异化处理**：重要的权重用高精度，不重要的用低精度
3. **优化效果**：在相同存储空间下获得更好的性能

**没有激活数据就像**：
- 闭着眼睛做菜
- 不知道调料的重要性
- 结果肯定不好

**有激活数据就像**：
- 边尝边调整
- 知道每种调料的作用
- 能做出好菜



## 如何选择对应激活数据

### **1.1 任意数据的可行性**

**技术上完全可行：**
- 用英文小说、新闻、维基百科等任意文本
- 用其他领域的数据（如医学、法律等）
- 甚至用随机生成的文本

**为什么可行：**
- 量化算法只需要激活值的统计信息
- 任何文本都能激活模型的神经元
- 从纯技术角度，有激活就能量化



### **1.2 不同数据选择的实际影响**

**影响程度排序（非实验验证）：**

| 数据类型 | 量化效果 | 性能保持 | 推荐程度 |
|---------|----------|----------|----------|
| **目标领域数据** | 最佳 | 90-95% | ✅✅✅ |
| **相关领域数据** | 良好 | 85-90% | ✅✅ |
| **通用高质量数据** | 一般 | 80-85% | ✅ |
| **无关领域数据** | 较差 | 70-80% | ⚠️ |
| **低质量/随机数据** | 差 | 60-70% | ❌ |



## 具体的数据选择分析

### **2.1 使用训练数据**

**优势：**
- **激活模式匹配**：校准时的激活分布与实际使用时高度一致
- **特征覆盖全面**：涵盖了模型学到的所有重要模式
- **量化精度最高**：统计信息最准确

```python
# 最优选择：GRPO数据
grpo_prompts = [
    "请分析当前股市的投资机会",
    "如何评估一只股票的价值？",
    "什么是价值投资策略？"
]

# 次优选择：SFT数据
sft_qa_pairs = [
    "用户：请解释股票投资的基本原理\n助手：股票投资的基本原理包括...",
    "用户：如何进行风险控制？\n助手：风险控制的方法有..."
]
```



### **2.2 使用专业领域数据**

**金融领域专业数据示例：**
```python
# 专业文章片段
finance_articles = [
    "央行货币政策的调整对股市的影响主要体现在...",
    "价值投资理论认为股票的内在价值由企业的基本面决定...",
    "技术分析通过研究价格走势和成交量来预测..."
]

# 专业问答
finance_qa = [
    "Q: 什么是市盈率？A: 市盈率是股票价格与每股收益的比值...",
    "Q: 如何分析财务报表？A: 财务报表分析包括..."
]
```

**效果分析：**
- **激活相关性**：能激活金融相关的神经元
- **词汇覆盖**：覆盖专业术语和概念
- **性能保持**：通常能保持85-90%的性能



### **2.3 使用通用数据**

**通用数据示例：**
```python
# C4数据集（常用选择）
c4_samples = [
    "The quick brown fox jumps over the lazy dog...",
    "Machine learning is a subset of artificial intelligence...",
    "Climate change refers to long-term shifts in global temperatures..."
]

# WikiText数据集
wiki_samples = [
    "Albert Einstein was a German-born theoretical physicist...",
    "The history of artificial intelligence began in antiquity..."
]
```

**效果分析：**
- **通用性好**：适用于大多数模型
- **获取容易**：标准数据集，无需自己准备
- **性能中等**：能保持80-85%的性能



### **2.4 量化效果对比举例**

**假设举例金融模型在不同校准数据下的表现：**

```python
# 模拟实验结果
calibration_results = {
    "GRPO原始数据": {
        "模型大小": "1.2GB → 0.35GB",
        "推理速度": "提升3.2x",
        "任务准确率": "92% → 88%",  # 下降4%
        "金融专业性": "完全保持"
    },
    "金融专业文章": {
        "模型大小": "1.2GB → 0.35GB", 
        "推理速度": "提升3.2x",
        "任务准确率": "92% → 85%",  # 下降7%
        "金融专业性": "基本保持"
    },
    "通用C4数据": {
        "模型大小": "1.2GB → 0.35GB",
        "推理速度": "提升3.2x", 
        "任务准确率": "92% → 78%",  # 下降14%
        "金融专业性": "明显下降"
    },
    "随机文本": {
        "模型大小": "1.2GB → 0.35GB",
        "推理速度": "提升3.2x",
        "任务准确率": "92% → 65%",  # 下降27%
        "金融专业性": "严重下降"
    }
}
```

### **2.5 为什么会有差异**

**技术原理：**
1. **激活分布差异**：不同数据激活不同的神经元组合
2. **统计偏差**：校准数据的分布与实际使用数据的分布不匹配
3. **量化误差累积**：统计信息不准确导致量化参数偏差

**具体影响：**
```python
# 金融数据 vs 通用数据的激活差异示例
financial_activations = {
    "financial_neurons": [0.8, 0.9, 0.7],  # 金融相关神经元高激活
    "general_neurons": [0.3, 0.4, 0.2]     # 通用神经元低激活
}

general_activations = {
    "financial_neurons": [0.2, 0.3, 0.1],  # 金融相关神经元低激活
    "general_neurons": [0.8, 0.9, 0.7]     # 通用神经元高激活
}
```

## 实践建议

**第一优先级：训练数据**
```python
# 最优组合
optimal_calibration = {
    "GRPO_prompts": 0.6,    # 60% - 最贴近实际使用
    "SFT_responses": 0.3,   # 30% - 增加激活多样性  
    "Finance_articles": 0.1  # 10% - 补充专业词汇
}
```

**第二优先级：领域专业数据**
```python
# 次优组合（如果没有训练数据）
fallback_calibration = {
    "Finance_QA": 0.5,      # 50% - 专业问答
    "Finance_articles": 0.3, # 30% - 专业文章
    "General_C4": 0.2       # 20% - 通用数据保底
}
```



## 总结说明

### **核心理解**

**可以用任意数据，但效果差别很大：**
- **理论上**：任何文本都能激活模型
- **实际上**：数据越匹配，量化效果越好

**最佳实践：**
1. **首选**：训练数据（GRPO + SFT）
2. **次选**：金融领域专业数据
3. **保底**：通用高质量数据（C4、WikiText）
4. **避免**：无关领域或低质量数据

### **5.2 实际建议**

**对于金融模型：**
- 主要使用GRPO问题数据
- 适当补充SFT数据增加多样性
- 如果数据不足，用金融专业文章补充
- 避免使用完全无关的数据

**记住：校准数据的选择直接影响量化后的模型性能！**

## 数据整理代码

## 1. calibration_fusion.py - 校准数据融合模块

### **技术原理**

**核心原理**：通过多源数据融合来构建更优质的校准数据集，提升量化效果
```python
# 数据融合的数学原理
final_dataset = Σ(dataset_i × ratio_i) where Σ(ratio_i) = 1
```

**智能检测算法**：
```python
def detect_file_type(self, file_path: str) -> str:
    # 通过数据结构特征自动识别格式
    # SFT格式：包含instruction、input、output字段
    # GRPO格式：包含prompt字段
    # 避免了手动指定格式的错误
```

### **设计关注点**

**关注点1：自适应数据类型检测**
```python
# 多重检测机制
1. 结构检测：分析JSON字段结构
2. 文件名检测：基于文件名模式匹配
3. 内容检测：分析实际数据内容
```

**优势**：
- 减少用户配置错误
- 支持混合格式数据源
- 提升数据处理的鲁棒性

**关注点2：差异化文本提取策略**
```python
# SFT数据提取：instruction+input → prompt, output → response
def extract_sft_texts(self, sft_file: str) -> List[Dict]:
    # 分别提取两种类型的文本
    prompt = f"{instruction}\n{input_text}"  # 合并为完整prompt
    output = data.get('output', '').strip()  # 保留<think>标签
    
# GRPO数据提取：直接提取prompt
def extract_grpo_texts(self, grpo_file: str) -> List[Dict]:
    # 专门提取强化学习的prompt数据
```

**技术优势**：
- **语义完整性**：保持instruction和input的语义连贯
- **标签保留**：保留思维链标签，提升量化质量
- **类型标记**：为每个样本标记数据源，便于后续分析

**关注点3：智能比例分配算法**
```python
# 自动比例归一化
if abs(sum(ratios) - 1.0) > 0.01:
    total_ratio = sum(ratios)
    ratios = [r / total_ratio for r in ratios]
    
# 按比例采样策略
target_samples = int(total_samples * ratios[i])
if len(texts) > target_samples:
    selected_texts = random.sample(texts, target_samples)
```

**算法优势**：
- **数值稳定**：避免浮点数精度问题
- **样本均衡**：确保各数据源按设定比例贡献
- **随机性控制**：通过种子保证结果可复现

### **实际应用价值**

**解决的核心问题**：
1. **数据质量不均**：通过智能融合提升整体数据质量
2. **格式多样性**：统一处理不同格式的训练数据
3. **比例控制**：精确控制不同数据源的贡献比例


```python  
# 数据代码实现
calibration_fusion.py 
```

```python 
#!/usr/bin/env python3
"""
量化校准数据融合工具 - JSON格式输出

这个工具的主要作用是：
1. 从不同格式的训练数据文件中提取文本
2. 将这些文本按指定比例融合成一个统一的校准数据集
3. 用于模型量化时的校准过程

支持的数据格式：
- SFT格式：包含instruction、input、output字段
- GRPO格式：包含prompt字段
"""

# 导入必要的库
import json       # 用于处理JSON格式的数据
import random     # 用于随机采样和打乱数据
import argparse   # 用于处理命令行参数
from pathlib import Path      # 用于处理文件路径
from typing import List, Dict # 用于类型注解，提高代码可读性

class CalibrationDataFusion:
    """
    校准数据融合类
    
    这个类的主要功能是将多个不同格式的训练数据文件
    融合成一个统一的校准数据集，用于模型量化
    """
    
    def __init__(self, random_seed: int = 42):
        """
        初始化方法
        
        参数说明：
        - random_seed: 随机种子，用于保证结果的可重复性
                      相同的种子会产生相同的随机结果
        """
        self.random_seed = random_seed  # 保存随机种子
        random.seed(random_seed)        # 设置随机种子，确保结果可重复
    
    def extract_sft_texts(self, sft_file: str) -> List[Dict]:
        """
        从SFT格式数据文件中提取文本
        
        SFT格式说明：
        - instruction: 指令文本
        - input: 输入文本  
        - output: 输出文本（可能包含<think>标签）
        
        参数说明：
        - sft_file: SFT格式数据文件的路径
        
        返回值：
        - 返回提取的文本列表，每个元素包含text、source、file字段
        """
        texts = []  # 用于存储提取的文本
        
        # 打开文件并逐行读取
        with open(sft_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    # 将每行JSON数据解析为Python字典
                    data = json.loads(line.strip())
                    
                    # 第一步：提取instruction和input，合并成prompt
                    instruction = data.get('instruction', '').strip()  # 获取指令，如果没有则为空字符串
                    input_text = data.get('input', '').strip()         # 获取输入，如果没有则为空字符串
                    
                    # 如果instruction和input都存在，则合并它们
                    if instruction and input_text:
                        prompt = f"{instruction}\n{input_text}"  # 用换行符连接
                        
                        # 只保留长度大于20的文本，过滤掉太短的无效文本
                        if len(prompt) > 20:
                            texts.append({
                                "text": prompt,           # 实际的文本内容
                                "source": "sft_prompt",   # 标记来源类型
                                "file": Path(sft_file).name  # 记录来源文件名
                            })
                    
                    # 第二步：提取output作为回复文本
                    output = data.get('output', '').strip()  # 获取输出文本
                    
                    # 如果output存在且长度足够，则保存
                    if output and len(output) > 20:
                        texts.append({
                            "text": output,              # 实际的文本内容
                            "source": "sft_response",    # 标记来源类型
                            "file": Path(sft_file).name  # 记录来源文件名
                        })
                        
                except Exception:
                    # 如果某行数据解析失败，跳过继续处理下一行
                    continue
        
        return texts  # 返回提取的所有文本
    
    def extract_grpo_texts(self, grpo_file: str) -> List[Dict]:
        """
        从GRPO格式数据文件中提取文本
        
        GRPO格式说明：
        - prompt: 提示文本，用于强化学习训练
        
        参数说明：
        - grpo_file: GRPO格式数据文件的路径
        
        返回值：
        - 返回提取的文本列表，每个元素包含text、source、file字段
        """
        texts = []  # 用于存储提取的文本
        
        # 打开文件并逐行读取
        with open(grpo_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    # 将每行JSON数据解析为Python字典
                    data = json.loads(line.strip())
                    
                    # 提取prompt字段
                    prompt = data.get('prompt', '').strip()  # 获取提示文本
                    
                    # 只保留长度大于20的文本，过滤掉太短的无效文本
                    if prompt and len(prompt) > 20:
                        texts.append({
                            "text": prompt,              # 实际的文本内容
                            "source": "grpo_prompt",     # 标记来源类型
                            "file": Path(grpo_file).name # 记录来源文件名
                        })
                        
                except Exception:
                    # 如果某行数据解析失败，跳过继续处理下一行
                    continue
        
        return texts  # 返回提取的所有文本
    
    def detect_file_type(self, file_path: str) -> str:
        """
        自动检测文件的数据格式类型
        
        检测逻辑：
        1. 优先通过文件内容的字段来判断
        2. 如果内容检测失败，则通过文件名来判断
        
        参数说明：
        - file_path: 要检测的文件路径
        
        返回值：
        - 'sft': SFT格式文件
        - 'grpo': GRPO格式文件  
        - 'unknown': 未知格式文件
        """
        # 方法1：通过文件内容检测
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    # 解析JSON数据
                    data = json.loads(line.strip())
                    
                    # 如果包含instruction和output字段，判断为SFT格式
                    if 'instruction' in data and 'output' in data:
                        return 'sft'
                    # 如果包含prompt字段，判断为GRPO格式
                    elif 'prompt' in data:
                        return 'grpo'
                except:
                    # 如果解析失败，继续检查下一行
                    continue
        
        # 方法2：通过文件名检测
        file_name = Path(file_path).name.lower()  # 获取文件名并转为小写
        
        if 'sft' in file_name:
            return 'sft'
        elif 'grpo' in file_name:
            return 'grpo'
        else:
            return 'unknown'  # 无法识别的格式
    
    def fuse_data(self, input_files: List[str], output_file: str, 
                  total_samples: int, ratios: List[float]) -> None:
        """
        融合多个数据文件的核心方法
        
        工作流程：
        1. 检测每个文件的格式类型
        2. 根据格式提取文本数据
        3. 按指定比例从每个文件中采样
        4. 将所有文本合并并随机打乱
        5. 保存为统一的JSON格式文件
        
        参数说明：
        - input_files: 输入文件路径列表
        - output_file: 输出文件路径
        - total_samples: 最终要生成的总样本数
        - ratios: 每个文件的采样比例列表
        """
        print(f"融合 {len(input_files)} 个文件，目标样本数: {total_samples}")
        
        # 第一步：归一化比例
        # 确保所有比例加起来等于1.0
        if abs(sum(ratios) - 1.0) > 0.01:  # 如果比例和不等于1
            total_ratio = sum(ratios)      # 计算当前比例总和
            ratios = [r / total_ratio for r in ratios]  # 重新计算比例
        
        all_texts = []  # 用于存储所有提取的文本
        
        # 第二步：处理每个输入文件
        for i, file_path in enumerate(input_files):
            print(f"处理文件: {Path(file_path).name}")
            
            # 检测文件类型
            file_type = self.detect_file_type(file_path)
            
            # 根据文件类型选择相应的提取方法
            if file_type == 'sft':
                texts = self.extract_sft_texts(file_path)
            elif file_type == 'grpo':
                texts = self.extract_grpo_texts(file_path)
            else:
                print(f"跳过未知格式文件: {file_path}")
                continue  # 跳过无法识别的文件
            
            # 第三步：按比例采样
            target_samples = int(total_samples * ratios[i])  # 计算当前文件应该提供的样本数
            
            if len(texts) > target_samples:
                # 如果文本数量超过目标，随机采样
                selected_texts = random.sample(texts, target_samples)
            else:
                # 如果文本数量不足，全部使用
                selected_texts = texts
                if len(texts) < target_samples:
                    print(f"警告：{file_path} 样本不足，实际获得 {len(texts)} 个")
            
            # 将选中的文本添加到总列表中
            all_texts.extend(selected_texts)
            print(f"已添加 {len(selected_texts)} 个样本")
        
        # 第四步：随机打乱并截取
        random.shuffle(all_texts)  # 随机打乱所有文本
        
        # 如果总数超过目标数量，截取到目标数量
        if len(all_texts) > total_samples:
            all_texts = all_texts[:total_samples]
        
        # 第五步：保存为JSON格式文件
        with open(output_file, 'w', encoding='utf-8') as f:
            for item in all_texts:
                # 每行写入一个JSON对象
                f.write(json.dumps(item, ensure_ascii=False) + '\n')
        
        # 输出完成信息
        print(f"融合完成！最终样本数: {len(all_texts)}")
        print(f"输出文件: {output_file}")

def main():
    """
    主函数 - 处理命令行参数并执行数据融合
    
    支持的命令行参数：
    - --input_files: 输入文件路径列表
    - --output_file: 输出文件路径
    - --total_samples: 总样本数
    - --ratios: 各文件的比例列表
    - --random_seed: 随机种子
    """
    # 创建命令行参数解析器
    parser = argparse.ArgumentParser(description='量化校准数据融合工具')
    
    # 添加各种命令行参数
    parser.add_argument('--input_files', '-i', nargs='+', required=True,
                        help='输入文件路径列表')
    parser.add_argument('--output_file', '-o', required=True,
                        help='输出文件路径')
    parser.add_argument('--total_samples', '-n', type=int, default=512,
                        help='总样本数 (默认: 512)')
    parser.add_argument('--ratios', '-r', nargs='+', type=float,
                        help='各文件的比例列表 (例如: 0.7 0.3)')
    parser.add_argument('--random_seed', '-s', type=int, default=42,
                        help='随机种子 (默认: 42)')
    
    # 解析命令行参数
    args = parser.parse_args()
    
    # 设置默认比例
    if args.ratios is None:
        if len(args.input_files) == 2:
            # 如果有2个文件，默认比例为7:3
            args.ratios = [0.7, 0.3]
        else:
            # 如果有多个文件，平均分配比例
            ratio = 1.0 / len(args.input_files)
            args.ratios = [ratio] * len(args.input_files)
    
    # 检查参数有效性
    if len(args.ratios) != len(args.input_files):
        print("错误：比例数量必须与输入文件数量相同")
        return
    
    # 执行数据融合
    fusion = CalibrationDataFusion(random_seed=args.random_seed)
    fusion.fuse_data(args.input_files, args.output_file, args.total_samples, args.ratios)

# 如果直接运行此脚本，则执行main函数
if __name__ == "__main__":
    main()

```

```bash
python calibration_fusion.py \
    --input_files deepspeek_sft_dataset_1_1k.jsonl grpo_prompts_dataset_5k.jsonl \
    --output_file calibration_data.jsonl \
    --total_samples 512 \
    --ratios 0.7 0.3 \
    --random_seed 42
``` 

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716110146377.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716105459566.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716110007618.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716110100886.png" width=100%></div>

# 3. 量化实现

### **项目整体分析**

### **1. 项目定位与核心价值**

**项目定位**：专门针对金融领域GRPO模型的GPTQ INT4量化完整解决方案

**核心价值**：
- 不是通用的量化工具，而是针对金融场景的专业化实现
- 从量化、评估到部署的完整工程化流程
- 考虑了金融领域的特殊需求和约束条件



### **2. 项目特色与关注点**

**技术特色**：

1. **专业化校准数据策略**：
   - 智能数据类型检测（SFT vs GRPO）
   - 多源数据融合算法
   - 基于业务场景的数据配比优化

2. **三维评估体系**：
   - 输出一致性评估：语义相似度计算
   - 稳定性评估：多次生成的一致性检验
   - 性能评估：精确的GPU计时和吞吐量分析

3. **生产级部署方案**：
   - vLLM高性能推理引擎集成
   - 分布式多GPU部署支持
   - 自动化部署脚本

**工程关注点**：

1. **完整的工程化流程**：
   ```json
   数据准备 → 模型量化 → 效果评估 → 性能测试 → 部署上线
   ```

2. **详尽的代码注释**：
   - 每个函数都有详细的参数说明
   - 核心算法有原理解释
   - 适合初学者学习




### **3. 项目架构分析**

**代码架构**：

```json
gptq_model/
├── 数据处理层
│   └── calibration_fusion.py          # 校准数据融合
├── 量化实现层
│   └── quantize_model.py               # 核心量化逻辑
├── 评估测试层
│   ├── financial_model_evaluation.py  # 综合评估系统
│   └── test_quantized_model.py        # 简单功能测试
└── 部署服务层
    ├── vllm_run.sh                    # vLLM服务部署
    └── deploy_A800.sh                 # 服务器部署
```



**技术架构**：

```json
输入层：原始模型 + 校准数据
     ↓
处理层：GPTQModel量化框架
     ↓
评估层：三维评估体系
     ↓
输出层：量化模型 + 评估报告
     ↓
部署层：vLLM推理服务
```



**数据流架构**：

```json
多源数据 → 数据融合 → 校准数据 → 量化过程 → 量化模型
                                    ↓
评估数据 → 效果评估 → 评估报告 → 部署决策 → 线上服务
```



### **5. 学习路径建议**

**第一阶段：理解基础概念**
1. 学习量化基础原理
2. 理解GPTQ算法核心思想
3. 掌握校准数据的作用机制

**第二阶段：分析核心实现**
1. 深入研究 `quantize_model.py`
   - 重点理解 `FinanceModelQuantizer` 类
   - 学习量化配置参数的含义
   - 掌握量化流程的每个步骤

2. 学习校准数据处理
   - 研究 `calibration_fusion.py`
   - 理解不同数据源的融合策略
   - 学习数据预处理的最佳实践

**第三阶段：掌握评估体系**
1. 学习 `financial_model_evaluation.py`
   - 理解三维评估框架设计
   - 掌握语义相似度计算方法
   - 学习评估报告生成逻辑

**第四阶段：实践部署流程**
1. 学习部署脚本
   - 理解 `vllm_run.sh` 的配置
   - 掌握分布式部署的要点
   - 学习服务化部署的最佳实践

2. 动手实践
   - 准备自己的校准数据
   - 执行完整的量化流程
   - 部署和测试量化模型



## quantize_model.py - 核心量化模块

### **技术原理**

**GPTQ算法原理**：
```python
# 基于Hessian矩阵的二阶优化量化
# 核心公式：minimize ||W - Ŵ||²_H
# 其中H是Hessian矩阵，W是原始权重，Ŵ是量化权重
```

**量化配置核心**：
```python
def setup_quantization_config(self):
    return QuantizeConfig(
        bits=4,                    # 目标位数：4位整数
        group_size=128,            # 分组大小：影响精度和速度平衡
        damp_percent=0.01,         # 阻尼系数：提升数值稳定性
        desc_act=False,            # 描述符激活：权衡精度和速度
    )
```

### **设计关注点**

**关注点1：专业级量化配置**
```python
# 参数详解及原理
bits=4                  # 最终存储位数
    # 原理：4位可表示16个不同值，相比32位节省87.5%空间
    # 权衡：空间节省 vs 精度损失

group_size=128          # 量化分组大小
    # 原理：每128个权重共享量化参数
    # 较小值(64)：更高精度，更多量化参数
    # 较大值(256)：更少参数，更高压缩比

damp_percent=0.01       # 阻尼系数
    # 作用：提升数值稳定性，避免量化过程中的数值问题
```

**关注点2：渐进式量化流程**
```python
def quantize_model(self, output_path: str):
    # 1. 数据准备阶段
    calibration_data = self.load_calibration_data()
    
    # 2. 配置设置阶段
    quantize_config = self.setup_quantization_config()
    
    # 3. 模型加载阶段
    self.model = GPTQModel.from_pretrained(...)
    
    # 4. 量化执行阶段
    self.model.quantize(calibration_data)
    
    # 5. 结果保存阶段
    self.model.save_quantized(output_path)
```


**关注点3：智能内存管理**
```python
# 精确的数据类型选择
torch_dtype=torch.float16    # 而非float32
    # 原理：量化过程中使用float16计算
    # 优势：节省50%内存，加速计算
    # 注意：最终存储仍为int4
```

**校准数据格式统一**
```python
def load_calibration_data(self) -> List[str]:
    # 支持多种数据格式的统一处理
    # JSON Lines格式 → 纯文本列表
    # 自动文本长度过滤和清理
    texts = [item["text"] for item in data if len(item["text"]) > 10]
```

**模型大小对比系统**
```python
def compare_model_sizes(self, quantized_path: str):
    # 精确的文件大小计算
    # 自动单位转换(GB, MB, KB)
    # 压缩比和节省空间的直观展示
```


pip install gptqmodel[all] -i https://pypi.tuna.tsinghua.edu.cn/simple

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250711162013441.png" width=100%></div>

**模型量化代码**

```python
#!/usr/bin/env python3
"""
GPTQModel INT4量化脚本 - 针对金融GRPO模型

这个脚本的主要作用：
1. 将原始的大语言模型从FP16/BF16精度压缩到INT4精度
2. 使用GPTQ算法进行量化，保持模型性能的同时显著减少模型大小
3. 量化后的模型可以在相同硬件上运行更大的模型或获得更快的推理速度

核心概念：
- 量化：将模型权重从32位/16位浮点数转换为4位整数，大幅减少存储空间
- 校准数据：用于统计模型激活分布的样本数据，帮助确定最佳量化参数
- GPTQ：一种先进的量化算法，通过二阶信息优化量化误差
"""

# 导入必要的库
import torch                                         # PyTorch深度学习框架
import json                                          # JSON数据处理
from transformers import AutoTokenizer               # 用于处理文本tokenization
from gptqmodel import GPTQModel, QuantizeConfig     # GPTQModel量化框架
from pathlib import Path                             # 文件路径处理
import time                                          # 时间计算

class FinanceModelQuantizer:
    """
    金融模型量化器类
    
    主要功能：
    1. 加载原始模型和校准数据
    2. 配置量化参数
    3. 执行量化过程
    4. 保存量化后的模型
    5. 对比模型大小变化
    """
    
    def __init__(self, model_path: str, calibration_data_path: str):
        """
        初始化量化器
        
        参数说明：
        model_path: 原始模型的路径，包含模型权重和配置文件
        calibration_data_path: 校准数据文件路径，用于量化过程中的统计信息收集
        """
        self.model_path = model_path                     # 存储原始模型路径
        self.calibration_data_path = calibration_data_path  # 存储校准数据路径
        self.tokenizer = None                            # 用于文本处理的分词器，初始化为空
        self.model = None                                # 用于存储加载的模型，初始化为空
        
    def load_calibration_data(self) -> list:
        """
        加载校准数据
        
        校准数据的作用：
        1. 量化过程中，需要通过校准数据来统计模型各层的激活分布
        2. 根据激活分布来确定最佳的量化参数（如缩放因子、零点等）
        3. 校准数据应该代表模型的实际使用场景，这样量化后的模型才能保持性能
        
        数据格式要求：
        - JSONL格式，每行一个JSON对象
        - 每个对象必须包含'text'字段
        - 文本长度建议在10-2048个字符之间
        
        返回值：
        calibration_data: 校准文本列表，每个元素是一个用于校准的文本字符串
        
        异常处理：
        - 跳过格式错误的行
        - 过滤掉过短的文本
        - 如果没有有效数据会返回空列表
        """
        print("正在加载校准数据...")
        calibration_data = []  # 存储校准文本的列表
        
        # 逐行读取校准数据文件
        with open(self.calibration_data_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    # 解析JSON格式的每一行
                    data = json.loads(line.strip())
                    # 提取文本内容，去除首尾空白字符
                    text = data.get('text', '').strip()
                    # 过滤掉太短的文本，确保校准数据质量
                    # 长度阈值设为10，避免无意义的短文本影响量化效果
                    if text and len(text) > 10:
                        calibration_data.append(text)
                except:
                    # 如果某行数据格式错误，跳过该行继续处理
                    continue
        
        print(f"成功加载了 {len(calibration_data)} 条校准数据")
        return calibration_data
    
    def setup_quantization_config(self) -> QuantizeConfig:
        """
        设置量化配置
        
        量化配置参数详解：
        - bits: 量化位数，4表示将权重量化为4位整数
        - group_size: 分组大小，128表示每128个权重为一组共享量化参数
                     较小的值(64)精度更高但速度慢，较大的值(256)速度快但精度略低
        - damp_percent: 阻尼系数，0.01表示在优化过程中的正则化强度
                       防止量化过程中的数值不稳定，通常设为0.01
        - desc_act: 是否启用描述符激活，False表示不启用（提升速度）
                   启用会提高精度但降低推理速度，金融场景通常关闭
        - static_groups: 是否使用静态分组，False表示动态分组
                        动态分组可以获得更好的量化效果
        - sym: 是否使用对称量化，True表示使用对称量化（更简单）
               对称量化计算更快，非对称量化精度略高
        - true_sequential: 是否使用真正的顺序量化，True表示按层顺序量化
                          确保量化过程的稳定性和可重复性
        
        返回值：
        quantize_config: 配置好的量化参数对象
        """
        print("正在设置量化配置...")
        
        quantize_config = QuantizeConfig(
            bits=4,                    # 量化位数：4位整数，相比16位浮点数大幅减少存储
            group_size=128,            # 分组大小：每128个权重共享一个量化参数，平衡精度和压缩率
            damp_percent=0.01,         # 阻尼系数：较小的值(0.01)减少量化过程中的数值震荡
            desc_act=False,            # 描述符激活：关闭以提升推理速度，对精度影响较小
            static_groups=False,       # 静态分组：关闭以允许动态优化分组策略
            sym=True,                  # 对称量化：启用对称量化，简化计算过程
            true_sequential=True,      # 顺序量化：按模型层的顺序进行量化，确保稳定性
        )
        
        print("量化配置设置完成")
        return quantize_config
    
    def quantize_model(self, output_path: str):
        """
        执行模型量化的主要流程
        
        参数说明：
        output_path: 量化后模型的保存路径
                    会自动创建目录结构，保存模型权重和配置文件
        
        量化流程：
        1. 加载校准数据 -> 2. 设置量化配置 -> 3. 加载原始模型 
        -> 4. 执行量化计算 -> 5. 保存量化模型 -> 6. 对比模型大小
        
        内存管理：
        - 使用torch.float16减少内存占用
        - 采用device_map="auto"自动分配GPU/CPU
        - 量化过程中可能需要大量内存，建议至少32GB
        
        时间预估：
        - 7B模型通常需要10-30分钟
        - 13B模型通常需要30-60分钟
        - 具体时间取决于硬件配置和校准数据量
        """
        print(f"开始量化模型: {self.model_path}")
        
        # 步骤1: 加载校准数据
        # 校准数据用于统计模型在实际数据上的激活分布
        calibration_data = self.load_calibration_data()
        
        # 步骤2: 设置量化配置
        # 配置量化算法的各种参数
        quantize_config = self.setup_quantization_config()
        
        # 步骤3: 加载分词器
        # 分词器用于将文本转换为模型可理解的token序列
        print("正在加载分词器...")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        
        # 步骤4: 加载原始模型
        # 使用GPTQModel框架加载模型，准备进行量化
        print("正在加载原始模型...")
        self.model = GPTQModel.from_pretrained(
            self.model_path,                    # 模型路径
            quantize_config=quantize_config,    # 量化配置
            device_map="auto",                  # 自动分配设备（CPU/GPU）
            torch_dtype=torch.float16,          # 使用float16精度以节省内存
            trust_remote_code=True              # 允许执行模型中的自定义代码
        )
        
        # 步骤5: 执行量化过程
        # 这是最耗时的步骤，会计算量化参数并转换模型权重
        print("正在进行量化计算...")
        start_time = time.time()  # 记录开始时间
        
        # 核心量化步骤：使用校准数据来优化量化参数
        # 这个过程会：
        # 1. 将校准数据输入模型获取激活值
        # 2. 统计每层的激活分布
        # 3. 计算最优的量化参数（缩放因子、零点）
        # 4. 将FP16权重转换为INT4权重
        self.model.quantize(calibration_data)
        
        end_time = time.time()  # 记录结束时间
        print(f"量化计算完成！耗时: {end_time - start_time:.2f} 秒")
        
        # 步骤6: 保存量化模型
        # 将量化后的模型和分词器保存到指定路径
        print(f"正在保存量化模型到: {output_path}")
        
        # 创建输出目录（如果不存在）
        Path(output_path).mkdir(parents=True, exist_ok=True)
        
        self.model.save_quantized(output_path)      # 保存量化后的模型权重
        self.tokenizer.save_pretrained(output_path)  # 保存分词器配置
        
        # 步骤7: 对比模型大小
        # 展示量化前后的模型大小变化
        self.compare_model_sizes(output_path)
        
        print("模型量化流程全部完成！")
    
    def compare_model_sizes(self, quantized_path: str):
        """
        对比原始模型和量化模型的大小
        支持 .bin 和 .safetensors 格式
        
        参数说明：
        quantized_path: 量化模型的保存路径
        
        支持格式：
        - .bin: 传统的PyTorch模型格式
        - .safetensors: 新的安全张量格式，加载更快更安全
        
        计算指标：
        - 绝对大小（GB）
        - 压缩比（倍数）
        - 节省空间（GB）
        - 压缩率（百分比）
        """
        try:
            # 支持多种模型文件格式
            def calculate_model_size(path):
                """
                计算模型文件总大小
                
                参数说明：
                path: 模型目录路径
                
                返回值：
                total_size: 总大小（字节）
                bin_count: .bin文件数量
                safetensors_count: .safetensors文件数量
                """
                total_size = 0
                path_obj = Path(path)
                
                # 查找 .bin 文件（传统PyTorch格式）
                bin_files = list(path_obj.rglob('*.bin'))
                total_size += sum(f.stat().st_size for f in bin_files if f.is_file())
                
                # 查找 .safetensors 文件（新格式，更安全）
                safetensors_files = list(path_obj.rglob('*.safetensors'))
                total_size += sum(f.stat().st_size for f in safetensors_files if f.is_file())
                
                return total_size, len(bin_files), len(safetensors_files)
            
            # 计算原始模型大小
            original_size, orig_bin_count, orig_safetensors_count = calculate_model_size(self.model_path)
            
            # 计算量化模型大小
            quantized_size, quant_bin_count, quant_safetensors_count = calculate_model_size(quantized_path)
            
            print(f"\n文件格式检测:")
            print(f"原始模型: {orig_bin_count} 个 .bin 文件, {orig_safetensors_count} 个 .safetensors 文件")
            print(f"量化模型: {quant_bin_count} 个 .bin 文件, {quant_safetensors_count} 个 .safetensors 文件")
            
            # 检查是否找到了模型文件
            if original_size == 0:
                print("警告: 未找到原始模型文件")
                print("请检查模型路径是否正确")
                return
            
            if quantized_size == 0:
                print("警告: 未找到量化模型文件")
                print("量化过程可能未成功完成")
                return
            
            # 转换为GB单位，方便阅读
            original_gb = original_size / (1024**3)        # 将字节转换为GB
            quantized_gb = quantized_size / (1024**3)      # 将字节转换为GB
            compression_ratio = original_size / quantized_size  # 计算压缩比
            
            # 显示详细的对比结果
            print(f"\n模型大小对比:")
            print(f"原始模型: {original_gb:.2f} GB")
            print(f"量化模型: {quantized_gb:.2f} GB")
            print(f"压缩比: {compression_ratio:.2f}倍")
            print(f"节省空间: {(original_gb - quantized_gb):.2f} GB")
            print(f"压缩率: {((original_size - quantized_size) / original_size * 100):.2f}%")
            
        except Exception as e:
            # 如果计算过程中出现错误，显示详细的警告信息
            print(f"无法计算模型大小对比: {e}")

def main():
    """
    主函数：配置路径并执行量化流程
    
    使用说明：
    1. 修改model_path为原始模型路径
    2. 修改calibration_data_path为校准数据路径
    3. 修改output_path为你想要保存量化模型的路径
    4. 运行脚本即可开始量化过程
    """
    # 配置路径
    model_path = "/shared/grpo_financial_tuning/output/best_model"  # 原始模型路径
    calibration_data_path = "calibration_data.jsonl"  # 校准数据路径
    output_path = "./output/best_model_gptq_int4"     # 量化模型输出路径
    
    # 检查路径是否存在
    if not Path(model_path).exists():
        print(f"错误：模型路径不存在: {model_path}")
        return
    
    if not Path(calibration_data_path).exists():
        print(f"错误：校准数据路径不存在: {calibration_data_path}")
        return
    
    # 创建量化器并执行量化
    quantizer = FinanceModelQuantizer(model_path, calibration_data_path)
    quantizer.quantize_model(output_path)

if __name__ == "__main__":
    main()

```

```python
# 执行命令
python quantize_model.py
```



<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716112206759.png" width=100%></div>


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716111907910.png" width=100%></div>


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716111927860.png" width=100%></div>


## 主要差异分析

### 1. **文件数量变化**
- **原始**: 9个模型文件
- **量化**: 2个模型文件
- **原因**: 量化过程中会重新组织模型结构，将权重重新分布到更少的文件中

### 2. **存储空间差异**
- **原始**: 15GB (FP16精度)
- **量化**: 5.3GB (INT4精度)
- **压缩比**: 2.83倍 (15GB ÷ 5.3GB)
- **节省空间**: 9.7GB (64.7%的空间节省)

### 3. **新增文件**
- **quantize_config.json**: 记录量化配置参数
- **quant_log.csv**: 记录量化过程的详细日志



## 差异产生的技术原因

### 1. **数据精度变化**
```
原始模型: FP16 (16位浮点数)
量化模型: INT4 (4位整数) + 量化参数

理论压缩比: 16 ÷ 4 = 4倍
实际压缩比: 2.83倍 (因为需要额外存储量化参数)
```

### 2. **权重存储方式**
```json
原始模型权重存储:
- 每个权重: 16位浮点数
- 直接存储原始数值

量化模型权重存储:
- 每个权重: 4位整数
- 额外存储: 缩放因子 + 零点偏移
- 分组量化: 每128个权重共享一套量化参数
```

### 3. **文件重新组织**
- **原始**: 按照原始模型层结构分片
- **量化**: 按照量化后的存储效率重新分片
- **结果**: 文件数量减少，每个文件大小更优化


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716111947713.png" width=100%></div>


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716112006555.png" width=100%></div>


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250711165510695.png" width=100%></div>


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716112033878.png" width=100%></div>

## 日志字段含义解析

### 字段说明
```json
layer: 模型层编号 (0-27，共28层)
module: 具体模块类型
  - self_attn.k_proj: 注意力机制的Key投影层
  - self_attn.v_proj: 注意力机制的Value投影层  
  - self_attn.q_proj: 注意力机制的Query投影层
  - self_attn.o_proj: 注意力机制的Output投影层
  - mlp.up_proj: MLP的上投影层
  - mlp.gate_proj: MLP的门控投影层
  - mlp.down_proj: MLP的下投影层
loss: 量化误差损失值
samples: 固定为0.01000
damp: 阻尼系数
time: 量化该模块耗时（秒）
```

## 关键数据分析

### 1. **量化损失分析**

**注意力模块损失 (较低，质量好)**
```json
K/V/Q/O投影层损失范围: 0.03-90.73
- 早期层(0-10): 损失很小 (0.03-7.87)
- 中期层(11-20): 损失适中 (1.41-22.35)
- 后期层(21-27): 损失较大 (2.37-95.39)
```

**MLP模块损失 (较高，但正常)**
```json
MLP层损失范围: 0.12-2792.19
- up_proj: 20.90-354.35
- gate_proj: 29.11-330.98  
- down_proj: 0.12-2792.19 (变化最大)
```

### 2. **损失分布规律**

**层级趋势**
- **浅层** (0-5): 损失较小，量化容易
- **中层** (6-20): 损失中等，量化稳定  
- **深层** (21-27): 损失增大，量化困难

**模块类型规律**
- **V投影**: 损失最小 (0.03-33.84)
- **K投影**: 损失较小 (0.17-4.85)
- **Q投影**: 损失中等 (0.77-53.74)
- **O投影**: 损失递增 (0.15-95.39)
- **MLP**: 损失最大，特别是down_proj

# 4. 量化效果检验


##  test_quantized_model.py - 简单测试模块

### **设计关注点**

**快速验证**
```python
# 最小化的功能测试
model = GPTQModel.from_quantized(model_path, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 直接的功能验证
response = model.generate(...)
print(f"模型回答: {response}")
```

**价值**：
- **快速反馈**：量化完成后立即验证功能
- **调试工具**：快速定位量化过程中的问题
```

test_quantized_model.py  最简测试

```python
"""
量化模型简单测试脚本

这个脚本的主要作用：
1. 快速测试量化模型是否正常工作
2. 验证模型加载和推理功能
3. 提供基本的模型调用示例
4. 用于开发和调试阶段的功能验证

使用场景：
- 量化完成后的功能验证
- 调试模型加载问题
- 测试不同的推理参数
- 验证模型输出质量
"""

import torch
from gptqmodel import GPTQModel        # 量化模型加载库
from transformers import AutoTokenizer  # 文本分词库

# 模型路径配置
model_path = "./output/best_model_gptq_int4"  # 量化模型的保存路径

# 加载量化模型
# GPTQModel.from_quantized用于加载已经量化的模型
print("正在加载量化模型...")
model = GPTQModel.from_quantized(
    model_path,              # 模型路径：量化模型的文件夹路径
    device_map="auto"        # 设备映射：自动分配CPU/GPU资源
)

# 加载对应的分词器
# 分词器用于将文本转换为模型可理解的token序列
print("正在加载分词器...")
tokenizer = AutoTokenizer.from_pretrained(model_path)

# 测试推理
prompt = "你是一个专业的金融领域分析师请对以下问题进行详细解答：周三盘前交易时段，金融板块呈现温和上涨态势：金融精选行业SPDR基金(XLF)上涨0.4%，Direxion每日三倍做多金融股ETF(FAS)上涨1.2%，而其反向产品Direxion每日三倍做空金融股ETF(FAZ)下跌1.1%。万事达卡(MA)股价上涨0.7%，因其董事会批准了最高110亿美元的普通股回购计划，并将季度股息提高16%。Cboe全球市场(CBOE)股价上涨0.5%，该公司披露其指数期权合约的11月日均交易量达到近400万份，较去年同期的330万份增长22%。基于这些市场动态，请分析以下问题：在当前市场环境下，影响金融板块ETF(XLF、FAS、FAZ)价格波动的关键驱动因素有哪些？如何评估万事达卡大规模股票回购和股息增长对其长期估值的影响？Cboe交易量的大幅增长可能预示着哪些市场结构变化？请综合考虑宏观经济环境、行业竞争格局和公司特定事件等多重因素展开分析。"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.7,
        do_sample=True
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"输入: {prompt}")
print(f"输出: {response}")
```

量化前模型
```python
python -c "
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 加载原始模型
model_path = '/shared/grpo_financial_tuning/output/best_model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map='auto', torch_dtype=torch.bfloat16)

# 测试推理
prompt = '你是一个专业的金融领域分析师请对以下问题进行详细解答：周三盘前交易时段，金融板块呈现温和上涨态势：金融精选行业SPDR基金(XLF)上涨0.4%，Direxion每日三倍做多金融股ETF(FAS)上涨1.2%，而其反向产品Direxion每日三倍做空金融股ETF(FAZ)下跌1.1%。万事达卡(MA)股价上涨0.7%，因其董事会批准了最高110亿美元的普通股回购计划，并将季度股息提高16%。Cboe全球市场(CBOE)股价上涨0.5%，该公司披露其指数期权合约的11月日均交易量达到近400万份，较去年同期的330万份增长22%。基于这些市场动态，请分析以下问题：在当前市场环境下，影响金融板块ETF(XLF、FAS、FAZ)价格波动的关键驱动因素有哪些？如何评估万事达卡大规模股票回购和股息增长对其长期估值的影响？Cboe交易量的大幅增长可能预示着哪些市场结构变化？请综合考虑宏观经济环境、行业竞争格局和公司特定事件等多重因素展开分析。'
inputs = tokenizer(prompt, return_tensors='pt').to(model.device)

with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7, do_sample=True)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f'输入: {prompt}')
print(f'输出: {response[len(prompt):].strip()}')
"
```


<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716155240995.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716155031183.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716120819370.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716120953534.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716121024757.png" width=100%></div>

**效果验证**

pip install sentence-transformers scikit-learn gptqmodel  -i https://pypi.tuna.tsinghua.edu.cn/simple

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250714192338669.png" width=100%></div>

## financial_model_evaluation.py - 综合评估模块

### **技术原理**

**三维评估理论框架**：
```python
评估维度 = {
    "consistency": 语义相似度评估,
    "stability": 输出稳定性评估,
    "performance": 推理性能评估
}
```

1. 输出一致性 (Output Consistency)
同样的问题 → 原始模型答案 vs 量化模型答案 → 比较相似度

2. 稳定性 (Stability)  
同样的问题问10次 → 10个答案之间的相似度 → 测试稳定性

3. 性能 (Performance)
同样的问题 → 原始模型用时 vs 量化模型用时 → 计算加速比

**语义相似度算法**：
```python
# 使用sentence-transformers计算语义相似度
similarity_score = cosine_similarity(
    embedding_original, 
    embedding_quantized
)
# 基于句子级别的语义表示，比词汇重叠更准确
```

### **设计关注点**

**关注点1：多模型并行评估架构**
```python
# 同时加载两个模型进行对比
self.original_model = AutoModelForCausalLM.from_pretrained(...)
self.quantized_model = GPTQModel.from_quantized(...)

# 智能内存管理
torch.cuda.empty_cache()  # 加载间清理GPU缓存
```

**技术优势**：
- **对比公平性**：相同环境下的直接对比
- **内存优化**：智能的GPU内存管理
- **并行效率**：避免重复加载的时间开销

**关注点2：精确的文本生成控制**
```python
def generate_response(self, model, tokenizer, prompt: str) -> str:
    # 输入长度限制
    max_input_length = 1536  # 为生成预留空间
    
    # 生成参数精确控制
    outputs = model.generate(
        max_new_tokens=self.config.max_new_tokens,
        temperature=self.config.temperature,
        do_sample=True,
        repetition_penalty=1.1,  # 减少重复
        top_p=0.9               # 核采样
    )
```

**参数优化原理**：
- **temperature=0.7**：平衡创造性和一致性
- **top_p=0.9**：保留90%概率质量，过滤低概率token
- **repetition_penalty=1.1**：轻微惩罚重复，保持自然性

**关注点3：多轮一致性测试**
```python
def evaluate_stability(self, prompts: List[str]) -> Dict:
    # 每个prompt生成多次响应
    responses = []
    for _ in range(self.config.num_consistency_trials):
        response = self.generate_response(model, tokenizer, prompt)
        responses.append(response)
    
    # 计算内部一致性
    consistency_score = self.calculate_self_consistency(responses)
```

**算法创新**：
- **自一致性度量**：同一输入多次生成的一致性
- **稳定性量化**：通过方差分析量化稳定性
- **异常检测**：识别异常输出并分析原因

### **评估方法论创新**

**创新1：语义级别的输出对比**
```python
# 不是简单的文本匹配，而是语义理解
def calculate_semantic_similarity(self, text1: str, text2: str) -> float:
    # 使用预训练的语义模型
    embeddings1 = self.similarity_model.encode([text1])
    embeddings2 = self.similarity_model.encode([text2])
    return cosine_similarity(embeddings1, embeddings2)[0][0]
```

**创新2：多维度性能分析**
```python
def evaluate_performance(self, prompts: List[str]) -> Dict:
    # 不只是速度，还包括吞吐量、内存使用等
    metrics = {
        "avg_latency": 平均延迟,
        "tokens_per_second": 每秒生成token数,
        "memory_usage": 内存使用情况,
        "gpu_utilization": GPU利用率
    }
```


```python
#!/usr/bin/env python3
"""
金融模型量化效果评估系统

这个脚本的主要作用：
1. 全面评估量化模型与原始模型的性能差异
2. 从三个维度进行评估：输出一致性、稳定性、推理性能
3. 生成详细的评估报告，帮助判断量化效果是否满足要求
4. 为模型部署决策提供数据支持

评估方法说明：
- 输出一致性：对比两个模型在相同输入下的输出相似度
- 稳定性：测试模型在多次生成中的一致性
- 推理性能：测试模型的推理速度和吞吐量
"""

import json
import random
import time
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
from gptqmodel import GPTQModel
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pathlib import Path
import logging
from typing import List, Dict, Tuple
from dataclasses import dataclass

# 配置日志系统：设置日志级别和输出格式
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class EvaluationConfig:
    """
    评估配置类
    
    这个类定义了评估过程中的所有配置参数
    使用dataclass装饰器自动生成构造函数和其他方法
    """
    # 模型路径配置
    original_model_path: str = "/shared/grpo_financial_tuning/output/best_model"      # 原始模型路径
    quantized_model_path: str = "/shared/gptq_model/output/best_model_gptq_int4"     # 量化模型路径
    
    # 数据和输出配置
    test_data_path: str = "/shared/gptq_model/grpo_prompts_dataset_5k.jsonl"         # 测试数据路径
    output_dir: str = "./evaluation_results"                                          # 结果输出目录
    
    # 评估参数配置
    num_test_samples: int = 10           # 测试样本数量：用于一致性评估的样本数
    num_consistency_trials: int = 10     # 一致性测试轮数：每个样本重复生成的次数
    max_new_tokens: int = 512           # 最大生成token数：控制生成文本的长度
    temperature: float = 0.7            # 温度参数：控制生成文本的随机性，0.7是平衡值
    random_seed: int = 42               # 随机种子：确保实验结果可复现
    warmup_rounds: int = 3              # 预热轮数：GPU预热次数，提高计时准确性

class FinancialModelEvaluator:
    """
    金融模型评估器主类
    
    功能概述：
    1. 加载原始模型和量化模型
    2. 执行三个维度的评估
    3. 生成详细的评估报告
    4. 保存评估结果
    """
    
    def __init__(self, config: EvaluationConfig):
        """
        初始化评估器
        
        参数说明：
        config: 评估配置对象，包含所有评估参数
        """
        self.config = config  # 存储配置对象
        
        # 语义相似度模型：用于计算文本之间的语义相似度
        self.similarity_model = SentenceTransformer('all-MiniLM-L6-v2')
        
        # 模型存储变量：初始化为None，在load_models方法中赋值
        self.original_model = None      # 原始模型
        self.quantized_model = None     # 量化模型
        self.original_tokenizer = None  # 原始模型的分词器
        self.quantized_tokenizer = None # 量化模型的分词器
        
        # 设置随机种子，确保结果可复现
        random.seed(config.random_seed)
        torch.manual_seed(config.random_seed)
        np.random.seed(config.random_seed)
        
        # 创建输出目录
        Path(config.output_dir).mkdir(exist_ok=True)
    
    def load_models(self):
        """
        加载原始模型和量化模型
        
        这是评估的第一步，需要将两个模型都加载到内存中
        加载过程包括：
        1. 加载分词器（将文本转换为token）
        2. 加载模型权重
        3. 设置合适的精度和设备分配
        """
        logger.info("开始加载模型...")
        
        try:
            # 加载原始模型
            logger.info("正在加载原始模型...")
            # 加载分词器：用于文本预处理
            self.original_tokenizer = AutoTokenizer.from_pretrained(self.config.original_model_path)
            # 加载模型：使用transformers库加载标准模型
            self.original_model = AutoModelForCausalLM.from_pretrained(
                self.config.original_model_path,  # 模型路径
                device_map="auto",                # 自动分配设备（CPU/GPU）
                torch_dtype=torch.bfloat16,       # 使用bfloat16精度节省内存
                trust_remote_code=True            # 允许执行自定义代码
            )
            logger.info("原始模型加载完成")
            
            # 清理GPU缓存，为加载第二个模型腾出空间
            torch.cuda.empty_cache()
            
            # 加载量化模型
            logger.info("正在加载量化模型...")
            # 加载量化模型的分词器
            self.quantized_tokenizer = AutoTokenizer.from_pretrained(self.config.quantized_model_path)
            # 加载量化模型：使用GPTQModel框架加载量化后的模型
            self.quantized_model = GPTQModel.from_quantized(
                self.config.quantized_model_path,  # 量化模型路径
                device_map="auto",                 # 自动分配设备
                torch_dtype=torch.bfloat16         # 使用bfloat16精度
            )
            logger.info("量化模型加载完成")
            
            # 记录模型设备分布信息
            self._log_model_device_info()
            
        except Exception as e:
            logger.error(f"模型加载失败: {e}")
            raise
    
    def _log_model_device_info(self):
        """
        记录模型设备分布信息
        
        功能：
        1. 显示GPU内存使用情况
        2. 帮助用户了解模型在设备上的分布
        3. 用于调试和优化内存使用
        """
        logger.info("=== 模型设备分布信息 ===")
        
        # 检查是否有可用的GPU
        if torch.cuda.is_available():
            # 遍历所有GPU设备
            for i in range(torch.cuda.device_count()):
                # 获取已分配的GPU内存（单位：字节）
                memory_allocated = torch.cuda.memory_allocated(i) / 1024**3  # 转换为GB
                # 获取已缓存的GPU内存（单位：字节）
                memory_cached = torch.cuda.memory_reserved(i) / 1024**3      # 转换为GB
                logger.info(f"GPU {i}: 已分配 {memory_allocated:.2f}GB, 已缓存 {memory_cached:.2f}GB")
    
    def load_test_data(self) -> List[str]:
        """
        加载测试数据
        
        功能：
        1. 从指定文件加载测试数据
        2. 过滤掉质量不好的数据
        3. 随机选择指定数量的样本
        4. 返回用于评估的prompt列表
        
        返回：
        prompts: 测试prompt列表，每个元素是一个字符串
        """
        logger.info(f"从 {self.config.test_data_path} 加载测试数据...")
        
        prompts = []  # 存储所有有效prompt的列表
        
        # 逐行读取测试数据文件
        with open(self.config.test_data_path, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    # 解析JSON格式的数据
                    data = json.loads(line.strip())
                    # 提取prompt字段
                    prompt = data.get('prompt', '').strip()
                    # 过滤条件：prompt不为空且长度大于50个字符
                    if prompt and len(prompt) > 50:
                        prompts.append(prompt)
                except:
                    # 如果数据格式错误，跳过该行
                    continue
        
        # 随机抽取指定数量的样本
        if len(prompts) > self.config.num_test_samples:
            prompts = random.sample(prompts, self.config.num_test_samples)
        
        logger.info(f"成功加载了 {len(prompts)} 个测试样本")
        return prompts
    
    def generate_response(self, model, tokenizer, prompt: str) -> str:
        """
        生成模型响应
        
        这是核心的文本生成方法，用于获取模型对给定prompt的响应
        
        参数说明：
        model: 要使用的模型（原始模型或量化模型）
        tokenizer: 对应的分词器
        prompt: 输入的提示文本
        
        返回：
        response: 模型生成的响应文本（去除了原始prompt）
        """
        # 将文本转换为模型输入格式
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        # 限制输入长度，避免超出模型最大长度限制
        max_input_length = 1536  # 为生成的token预留空间
        if inputs['input_ids'].shape[1] > max_input_length:
            # 如果输入过长，进行截断
            inputs['input_ids'] = inputs['input_ids'][:, :max_input_length]
            inputs['attention_mask'] = inputs['attention_mask'][:, :max_input_length]
        
        # 生成文本
        with torch.no_grad():  # 禁用梯度计算，节省内存
            outputs = model.generate(
                **inputs,                                    # 输入数据
                max_new_tokens=self.config.max_new_tokens,   # 最大生成token数
                temperature=self.config.temperature,         # 温度参数：控制随机性
                do_sample=True,                              # 启用采样：产生多样化的输出
                pad_token_id=tokenizer.eos_token_id,        # 填充token ID
                repetition_penalty=1.1,                     # 重复惩罚：减少重复内容
                top_p=0.9                                   # 核采样：保留概率前90%的token
            )
        
        # 解码生成的token为文本
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # 移除原始prompt，只保留生成的部分
        response = response[len(prompt):].strip()
        
        return response
    
    def calculate_semantic_similarity(self, text1: str, text2: str) -> float:
        """
        计算两个文本之间的语义相似度
        
        使用预训练的sentence-transformers模型计算语义相似度
        这比简单的文本匹配更能反映内容的实际相似性
        
        参数说明：
        text1: 第一个文本
        text2: 第二个文本
        
        返回：
        similarity: 相似度分数，范围[0,1]，1表示完全相似
        """
        try:
            # 将文本转换为语义向量
            embeddings1 = self.similarity_model.encode([text1])
            embeddings2 = self.similarity_model.encode([text2])
            
            # 计算余弦相似度
            similarity = cosine_similarity(embeddings1, embeddings2)[0][0]
            
            return float(similarity)
        except Exception as e:
            logger.warning(f"相似度计算失败: {e}")
            return 0.0
    
    def evaluate_output_consistency(self, prompts: List[str]) -> Dict:
        """
        评估输出一致性
        
        这是核心评估方法之一，用于测试量化模型和原始模型的输出相似度
        
        评估原理：
        1. 对每个prompt，让两个模型都生成响应
        2. 计算两个响应之间的语义相似度
        3. 统计所有样本的相似度分布
        4. 评估量化是否显著影响了模型输出
        
        参数说明：
        prompts: 测试prompt列表
        
        返回：
        consistency_stats: 包含相似度统计信息的字典
        """
        logger.info("=== 开始输出一致性评估 ===")
        
        similarities = []        # 存储所有相似度分数
        detailed_results = []    # 存储详细的对比结果
        
        # 对每个prompt进行评估
        for i, prompt in enumerate(prompts):
            logger.info(f"处理第 {i+1}/{len(prompts)} 个样本...")
            
            try:
                # 生成原始模型的响应
                original_response = self.generate_response(
                    self.original_model, self.original_tokenizer, prompt
                )
                
                # 生成量化模型的响应
                quantized_response = self.generate_response(
                    self.quantized_model, self.quantized_tokenizer, prompt
                )
                
                # 计算语义相似度
                similarity = self.calculate_semantic_similarity(original_response, quantized_response)
                similarities.append(similarity)
                
                # 记录详细结果（用于后续分析）
                detailed_results.append({
                    "prompt": prompt[:100] + "...",  # 截断prompt用于显示
                    "original_response": original_response,
                    "quantized_response": quantized_response,
                    "similarity": similarity
                })
                
                logger.info(f"相似度: {similarity:.3f}")
                
            except Exception as e:
                logger.error(f"处理样本 {i+1} 时出错: {e}")
                continue
        
        # 计算统计指标
        consistency_stats = {
            "mean_similarity": np.mean(similarities),      # 平均相似度
            "std_similarity": np.std(similarities),        # 相似度标准差
            "min_similarity": np.min(similarities),        # 最小相似度
            "max_similarity": np.max(similarities),        # 最大相似度
            "percentiles": {                               # 百分位数分析
                "25th": np.percentile(similarities, 25),   # 25%分位数
                "50th": np.percentile(similarities, 50),   # 中位数
                "75th": np.percentile(similarities, 75),   # 75%分位数
                "90th": np.percentile(similarities, 90)    # 90%分位数
            },
            "samples_processed": len(similarities),        # 成功处理的样本数
            "detailed_results": detailed_results           # 详细结果
        }
        
        logger.info(f"一致性评估完成，平均相似度: {consistency_stats['mean_similarity']:.3f}")
        return consistency_stats
    
    def evaluate_stability(self, test_prompts: List[str]) -> Dict:
        """
        评估模型稳定性（自一致性）
        
        稳定性评估的目的：
        1. 测试模型在相同输入下的输出一致性
        2. 评估量化是否增加了模型的不稳定性
        3. 比较原始模型和量化模型的稳定性保持程度
        
        评估方法：
        1. 选择部分prompt
        2. 每个prompt重复生成多次
        3. 计算多次生成结果之间的相似度
        4. 对比两个模型的稳定性
        
        参数说明：
        test_prompts: 测试prompt列表
        
        返回：
        stability_stats: 包含稳定性统计信息的字典
        """
        logger.info("=== 开始稳定性评估 ===")
        
        # 选择部分prompt进行稳定性测试（稳定性测试计算量大）
        stability_prompts = random.sample(test_prompts, min(10, len(test_prompts)))
        
        original_stability_scores = []   # 原始模型的稳定性分数
        quantized_stability_scores = []  # 量化模型的稳定性分数
        
        # 对每个prompt进行稳定性测试
        for i, prompt in enumerate(stability_prompts):
            logger.info(f"稳定性测试 {i+1}/{len(stability_prompts)}")
            
            # 测试原始模型的稳定性
            original_responses = []
            for trial in range(self.config.num_consistency_trials):
                response = self.generate_response(self.original_model, self.original_tokenizer, prompt)
                original_responses.append(response)
            
            # 计算原始模型的自一致性分数
            original_stability = self._calculate_self_consistency(original_responses)
            original_stability_scores.append(original_stability)
            
            # 测试量化模型的稳定性
            quantized_responses = []
            for trial in range(self.config.num_consistency_trials):
                response = self.generate_response(self.quantized_model, self.quantized_tokenizer, prompt)
                quantized_responses.append(response)
            
            # 计算量化模型的自一致性分数
            quantized_stability = self._calculate_self_consistency(quantized_responses)
            quantized_stability_scores.append(quantized_stability)
            
            logger.info(f"原始模型稳定性: {original_stability:.3f}, 量化模型稳定性: {quantized_stability:.3f}")
        
        # 计算稳定性统计指标
        stability_stats = {
            "original_model": {
                "mean_stability": np.mean(original_stability_scores),  # 原始模型平均稳定性
                "std_stability": np.std(original_stability_scores),    # 原始模型稳定性标准差
                "scores": original_stability_scores                    # 所有稳定性分数
            },
            "quantized_model": {
                "mean_stability": np.mean(quantized_stability_scores), # 量化模型平均稳定性
                "std_stability": np.std(quantized_stability_scores),   # 量化模型稳定性标准差
                "scores": quantized_stability_scores                   # 所有稳定性分数
            },
            # 稳定性保持率：量化模型稳定性 / 原始模型稳定性
            "stability_retention": np.mean(quantized_stability_scores) / np.mean(original_stability_scores)
        }
        
        logger.info(f"稳定性评估完成，稳定性保持率: {stability_stats['stability_retention']:.3f}")
        return stability_stats
    
    def _calculate_self_consistency(self, responses: List[str]) -> float:
        """
        计算自一致性分数
        
        自一致性的定义：同一个prompt多次生成的结果之间的相似度
        
        计算方法：
        1. 对所有响应进行两两相似度计算
        2. 计算所有相似度的平均值
        3. 返回平均相似度作为自一致性分数
        
        参数说明：
        responses: 同一个prompt的多次响应列表
        
        返回：
        self_consistency: 自一致性分数，范围[0,1]
        """
        # 如果响应数量少于2，无法计算相似度
        if len(responses) < 2:
            return 1.0
        
        similarities = []  # 存储所有相似度分数
        
        # 计算所有响应之间的两两相似度
        for i in range(len(responses)):
            for j in range(i + 1, len(responses)):
                sim = self.calculate_semantic_similarity(responses[i], responses[j])
                similarities.append(sim)
        
        # 返回平均相似度
        return np.mean(similarities)
    
    def evaluate_performance(self, test_prompts: List[str]) -> Dict:
        """
        评估推理性能
        
        性能评估的目的：
        1. 测试量化模型相比原始模型的速度提升
        2. 评估量化带来的实际性能收益
        3. 为部署决策提供性能数据
        
        评估方法：
        1. 分别测试两个模型的推理时间
        2. 计算加速比
        3. 进行统计分析
        
        参数说明：
        test_prompts: 测试prompt列表
        
        返回：
        performance_stats: 包含性能统计信息的字典
        """
        logger.info("=== 开始性能评估 ===")
        
        # 选择部分prompt进行性能测试
        performance_prompts = random.sample(test_prompts, min(10, len(test_prompts)))
        
        # 分别测试每个模型，避免交替影响
        original_times = []   # 原始模型推理时间列表
        quantized_times = []  # 量化模型推理时间列表
        
        # 测试原始模型性能
        logger.info("测试原始模型性能...")
        for i, prompt in enumerate(performance_prompts):
            logger.info(f"原始模型测试 {i+1}/{len(performance_prompts)}")
            
            # GPU预热：避免冷启动影响计时准确性
            inputs = self.original_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1536).to(self.original_model.device)
            with torch.no_grad():
                for _ in range(self.config.warmup_rounds):
                    self.original_model.generate(**inputs, max_new_tokens=50, do_sample=False)
            
            # 准确计时：使用GPU同步确保计时准确
            torch.cuda.synchronize()  # 等待GPU操作完成
            start_time = time.time()
            _ = self.generate_response(self.original_model, self.original_tokenizer, prompt)
            torch.cuda.synchronize()  # 等待GPU操作完成
            original_time = time.time() - start_time
            original_times.append(original_time)
        
        # 清理GPU缓存
        torch.cuda.empty_cache()
        
        # 测试量化模型性能
        logger.info("测试量化模型性能...")
        for i, prompt in enumerate(performance_prompts):
            logger.info(f"量化模型测试 {i+1}/{len(performance_prompts)}")
            
            # GPU预热
            inputs = self.quantized_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1536).to(self.quantized_model.device)
            with torch.no_grad():
                for _ in range(self.config.warmup_rounds):
                    self.quantized_model.generate(**inputs, max_new_tokens=50, do_sample=False)
            
            # 准确计时
            torch.cuda.synchronize()
            start_time = time.time()
            _ = self.generate_response(self.quantized_model, self.quantized_tokenizer, prompt)
            torch.cuda.synchronize()
            quantized_time = time.time() - start_time
            quantized_times.append(quantized_time)
        
        # 计算性能统计指标
        performance_stats = {
            "original_model": {
                "mean_time": np.mean(original_times),   # 原始模型平均时间
                "std_time": np.std(original_times),     # 原始模型时间标准差
                "times": original_times                 # 所有时间记录
            },
            "quantized_model": {
                "mean_time": np.mean(quantized_times),  # 量化模型平均时间
                "std_time": np.std(quantized_times),    # 量化模型时间标准差
                "times": quantized_times                # 所有时间记录
            },
            # 加速比：原始模型时间 / 量化模型时间
            "speedup": np.mean(original_times) / np.mean(quantized_times)
        }
        
        logger.info(f"原始模型平均时间: {performance_stats['original_model']['mean_time']:.2f}秒")
        logger.info(f"量化模型平均时间: {performance_stats['quantized_model']['mean_time']:.2f}秒")
        logger.info(f"加速比: {performance_stats['speedup']:.2f}x")
        
        return performance_stats
    
    def run_complete_evaluation(self) -> Dict:
        """
        运行完整的评估流程
        
        这是主要的评估方法，按顺序执行所有评估步骤：
        1. 加载模型
        2. 加载测试数据
        3. 执行三个维度的评估
        4. 生成综合报告
        5. 保存结果
        
        返回：
        results: 包含所有评估结果的字典
        """
        logger.info("=== 开始金融模型量化评估 ===")
        
        # 步骤1: 加载模型
        self.load_models()
        
        # 步骤2: 加载测试数据
        test_prompts = self.load_test_data()
        
        # 步骤3: 执行各项评估
        results = {
            "config": {
                "num_test_samples": len(test_prompts),
                "max_new_tokens": self.config.max_new_tokens,
                "temperature": self.config.temperature,
                "random_seed": self.config.random_seed
            },
            # 一致性评估：测试输出相似度
            "consistency_evaluation": self.evaluate_output_consistency(test_prompts),
            # 稳定性评估：测试输出稳定性
            "stability_evaluation": self.evaluate_stability(test_prompts),
            # 性能评估：测试推理速度
            "performance_evaluation": self.evaluate_performance(test_prompts)
        }
        
        # 步骤4: 保存结果
        self._save_results(results)
        
        # 步骤5: 生成评估报告
        self._generate_report(results)
        
        return results
    
    def _save_results(self, results: Dict):
        """
        保存评估结果到JSON文件
        
        参数说明：
        results: 包含所有评估结果的字典
        """
        output_file = Path(self.config.output_dir) / "evaluation_results.json"
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        logger.info(f"结果已保存到: {output_file}")
    
    def _generate_report(self, results: Dict):
        """
        生成评估报告
        
        根据评估结果生成人类可读的报告，包括：
        1. 基本配置信息
        2. 各项评估指标
        3. 综合评估结论
        4. 部署建议
        
        参数说明：
        results: 包含所有评估结果的字典
        """
        # 提取各项评估结果
        consistency = results["consistency_evaluation"]
        stability = results["stability_evaluation"]
        performance = results["performance_evaluation"]
        
        # 生成报告文本
        report = f"""
=== 金融模型量化评估报告 ===

评估配置:
- 测试样本数量: {results['config']['num_test_samples']}
- 生成token数: {results['config']['max_new_tokens']}
- 温度参数: {results['config']['temperature']}

输出一致性评估:
- 平均语义相似度: {consistency['mean_similarity']:.3f}
- 相似度标准差: {consistency['std_similarity']:.3f}
- 最低相似度: {consistency['min_similarity']:.3f}
- 90%分位数: {consistency['percentiles']['90th']:.3f}

稳定性评估:
- 原始模型自一致性: {stability['original_model']['mean_stability']:.3f}
- 量化模型自一致性: {stability['quantized_model']['mean_stability']:.3f}
- 稳定性保持率: {stability['stability_retention']:.3f}

性能评估:
- 原始模型平均推理时间: {performance['original_model']['mean_time']:.2f}秒
- 量化模型平均推理时间: {performance['quantized_model']['mean_time']:.2f}秒
- 推理速度提升: {performance['speedup']:.2f}倍

综合评估结论:
"""
        
        # 根据评估结果添加结论
        if consistency['mean_similarity'] > 0.85:
            report += "输出一致性: 优秀 (相似度 > 0.85)\n"
        elif consistency['mean_similarity'] > 0.8:
            report += "输出一致性: 良好 (相似度 > 0.8)\n"
        else:
            report += "输出一致性: 需要改进 (相似度 < 0.8)\n"
        
        if stability['stability_retention'] > 0.9:
            report += "稳定性保持: 优秀 (保持率 > 90%)\n"
        elif stability['stability_retention'] > 0.8:
            report += "稳定性保持: 良好 (保持率 > 80%)\n"
        else:
            report += "稳定性保持: 需要改进 (保持率 < 80%)\n"
        
        if performance['speedup'] > 2.0:
            report += "性能提升: 优秀 (加速比 > 2x)\n"
        elif performance['speedup'] > 1.5:
            report += "性能提升: 良好 (加速比 > 1.5x)\n"
        else:
            report += "性能提升: 不明显 (加速比 < 1.5x)\n"
        
        # 最终建议
        if (consistency['mean_similarity'] > 0.85 and 
            stability['stability_retention'] > 0.8 and 
            performance['speedup'] > 1.5):
            report += "\n建议: 量化效果良好，可以考虑部署到生产环境\n"
        else:
            report += "\n建议: 量化效果有待改进，建议调整量化参数或方法\n"
        
        print(report)
        
        # 保存报告到文件
        report_file = Path(self.config.output_dir) / "evaluation_report.txt"
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write(report)
        
        logger.info(f"报告已保存到: {report_file}")

def main():
    """
    主函数
    
    创建评估配置和评估器，执行完整的评估流程
    """
    # 创建评估配置
    config = EvaluationConfig(
        test_data_path="/shared/gptq_model/grpo_prompts_dataset_5k.jsonl",
        output_dir="./evaluation_results",
        num_test_samples=20,        # 可调整测试样本数
        warmup_rounds=5,            # 预热轮数
        random_seed=42
    )
    
    # 创建评估器
    evaluator = FinancialModelEvaluator(config)
    
    try:
        # 运行评估
        results = evaluator.run_complete_evaluation()
        logger.info("评估完成!")
        
    except Exception as e:
        logger.error(f"评估过程中出现错误: {e}")
        raise

if __name__ == "__main__":
    main()

```

```python 
 python financial_model_evaluation.py
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716122121728.png" width=100%></div>

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716121429523.png" width=100%></div>

# 5. 量化模型部署

```bash
#!/bin/bash

# vLLM量化模型部署脚本     注意删除注释使用、注意删除注释使用、注意删除注释使用
#
# 这个脚本的主要作用：
# 1. 启动Ray分布式计算框架（用于多GPU并行推理）
# 2. 部署量化模型为API服务
# 3. 提供OpenAI兼容的API接口
# 4. 支持高并发推理请求
#
# 使用场景：
# - 生产环境部署量化模型
# - 提供HTTP API接口供其他系统调用
# - 支持多用户并发访问
# - 实现模型推理服务化

# 基本配置参数
HEAD_NODE="10.60.68.220"                                    # 主节点IP地址：Ray集群的协调节点
MODEL_PATH="/shared/gptq_model/output/best_model_gptq_int4" # 量化模型路径：要部署的模型文件位置
PORT=8000                                                   # API服务端口：HTTP服务监听的端口号

echo "启动Ray集群"

# 激活conda环境
# conda是Python包管理器，这里激活特定的Python环境
source ~/miniconda3/etc/profile.d/conda.sh  # 加载conda命令
conda activate reward3                        # 激活名为reward3的conda环境

# 设置分布式训练环境变量
# 这些环境变量用于配置多机多卡的分布式计算
export MASTER_ADDR=$HEAD_NODE    # 主节点地址：分布式训练的协调节点
export MASTER_PORT=29500         # 主节点端口：用于节点间通信的端口
export GLOO_SOCKET_IFNAME=eth0   # 网络接口名：指定用于通信的网络接口
export NCCL_SOCKET_IFNAME=eth0   # NCCL通信接口：GPU间通信使用的网络接口

# 停止现有的Ray进程
# 确保没有残留的Ray进程影响新的启动
ray stop --force 2>/dev/null || true  # --force强制停止，2>/dev/null忽略错误输出

# 启动Ray头节点
# Ray是分布式计算框架，用于管理多GPU资源
ray start --head \              # 启动头节点模式
  --port=6379 \                 # Ray服务端口：默认6379
  --dashboard-host=0.0.0.0 \    # 监控面板地址：0.0.0.0表示允许所有IP访问
  --dashboard-port=8265 \       # 监控面板端口：Web界面端口
  --num-gpus=1 \                # GPU数量：当前节点的GPU数量
  --num-cpus=24                 # CPU数量：当前节点的CPU核心数

echo "启动vLLM单机服务器..."

# 启动vLLM API服务器
# vLLM是高性能推理引擎，支持量化模型推理
python -m vllm.entrypoints.openai.api_server \
  --model $MODEL_PATH \                    # 模型路径：要部署的量化模型
  --host 0.0.0.0 \                        # 监听地址：0.0.0.0表示接受所有IP的请求
  --port $PORT \                          # 服务端口：API服务的HTTP端口
  --tensor-parallel-size 1 \              # 张量并行度：单GPU设置为1
  --dtype bfloat16 \                      # 数据类型：使用bfloat16精度节省内存
  --max-model-len 2048 \                  # 最大序列长度：限制输入和输出的总长度
  --gpu-memory-utilization 0.8 \         # GPU内存利用率：使用80%的GPU内存
  --disable-log-requests \                # 禁用请求日志：减少日志输出
  --api-key gptq-model-key \              # API密钥：用于验证客户端身份
  --distributed-executor-backend ray      # 分布式后端：使用Ray管理多GPU资源

```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716124200155.png" width=100%></div>

```python 
# 测试API服务是否正常启动
# 使用Python发送HTTP请求测试API接口
python -c "
import requests
resp = requests.post('http://localhost:8000/v1/chat/completions', 
    headers={'Authorization': 'Bearer gptq-model-key'},
    json={'model': './output/best_model_gptq_int4', 'messages': [{'role': 'user', 'content': '你是一个专业的金融领域分析师请对以下问题进行详细解答：周三盘前交易时段，金融板块呈现温和上涨态势：金融精选行业SPDR基金(XLF)上涨0.4%，Direxion每日三倍做多金融股ETF(FAS)上涨1.2%，而其反向产品Direxion每日三倍做空金融股ETF(FAZ)下跌1.1%。万事达卡(MA)股价上涨0.7%，因其董事会批准了最高110亿美元的普通股回购计划，并将季度股息提高16%。Cboe全球市场(CBOE)股价上涨0.5%，该公司披露其指数期权合约的11月日均交易量达到近400万份，较去年同期的330万份增长22%。基于这些市场动态，请分析以下问题：在当前市场环境下，影响金融板块ETF(XLF、FAS、FAZ)价格波动的关键驱动因素有哪些？如何评估万事达卡大规模股票回购和股息增长对其长期估值的影响？Cboe交易量的大幅增长可能预示着哪些市场结构变化？请综合考虑宏观经济环境、行业竞争格局和公司特定事件等多重因素展开分析。'}], 'max_tokens': 1024})
print('Status Code:', resp.status_code)
print('Response:', resp.json())
"
```

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716123345322.png" width=100%></div>

# 6. 总结

<div align=center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/muyan/image-20250716171337199.png" width=100%></div>